In [15]:
from gatubelxs.page_views import generate_page_view
from gatubelxs.click_events import generate_click_event
from gatubelxs.user_interaction import generate_user_interaction
from kafka import KafkaProducer
from random import randint
import json
import time

In [16]:
KAFKA_SERVER = '3e6802a385a9:9093'
TOPIC_PAGE_VIEWS = "page_views"
TOPIC_CLICK_EVENTS = "click_events"
TOPIC_USER_INTERACTIONS = "user_interactions"

NUMBER_SESSION = 25 #Number of web sessions to simulate
MAX_EVENTS_PER_SESSION = 10

In [21]:
#Producer
producer = KafkaProducer(
    bootstrap_servers=KAFKA_SERVER,
    value_serializer=lambda m: json.dumps(m).encode('utf-8')
)

In [18]:
import uuid
from random import randint

def simulate_session():
    user_id = str(uuid.uuid4())
    session_id = str(uuid.uuid4())

    n_page_views = randint(1, MAX_EVENTS_PER_SESSION)
    n_clicks = randint(0, MAX_EVENTS_PER_SESSION)
    n_interactions = randint(0, MAX_EVENTS_PER_SESSION)

    for _ in range(n_page_views):
        evt = generate_page_view(user_id, session_id)
        producer.send(TOPIC_PAGE_VIEWS, evt)
        print(f"[{TOPIC_PAGE_VIEWS}] ->", evt)
    
    for _ in range(n_clicks):
        evt = generate_click_event(user_id, session_id)
        producer.send(TOPIC_CLICK_EVENTS, evt)
        print(f"[{TOPIC_CLICK_EVENTS}] ->", evt)
    
    for i in range(n_interactions):
        evt = generate_user_interaction(user_id, session_id)
        producer.send(TOPIC_USER_INTERACTIONS, evt)
        print(f"[{TOPIC_USER_INTERACTIONS}] ->", evt)

In [22]:
try:
    print(f"Starting to produce sessions...")
    for i in range(NUMBER_SESSION):
        simulate_session()
        time.sleep(2)
except KeyboardInterrupt:
    print("Interrumpted by user; stopping.")
finally:
    producer.flush()
    producer.close()

Starting to produce sessions...
[page_views] -> {'user_id': '3c113af1-f3a8-4aad-a7ad-3cfeaf0ec351', 'session_id': '812704cf-5400-4c67-9d90-0a7b529b1923', 'page_url': '/product/1127', 'referrer_url': 'tag', 'category': 'electronics', 'price': 46.86, 'timestamp': '2025-05-08T23:54:15.361448'}
[page_views] -> {'user_id': '3c113af1-f3a8-4aad-a7ad-3cfeaf0ec351', 'session_id': '812704cf-5400-4c67-9d90-0a7b529b1923', 'page_url': '/product/1146', 'referrer_url': 'explore/search', 'category': 'books', 'price': 121.11, 'timestamp': '2025-05-08T23:54:15.369936'}
[page_views] -> {'user_id': '3c113af1-f3a8-4aad-a7ad-3cfeaf0ec351', 'session_id': '812704cf-5400-4c67-9d90-0a7b529b1923', 'page_url': '/product/1148', 'referrer_url': 'app', 'category': 'home', 'price': 485.92, 'timestamp': '2025-05-08T23:54:15.370060'}
[page_views] -> {'user_id': '3c113af1-f3a8-4aad-a7ad-3cfeaf0ec351', 'session_id': '812704cf-5400-4c67-9d90-0a7b529b1923', 'page_url': '/product/1006', 'referrer_url': 'list/posts', 'catego